## Game Feature Recommendation System
Recommends to the user on prompting game features extracted from game database information

In [2]:
# Library imports

import numpy as np
import spacy
import csv
nlp = spacy.load("en_core_web_sm")

In [ ]:
# External script imports


### Steps: 
1. Load the data (tags => game features)
2. Sort each tag by the co-occurence matrix
3. Get a user prompt for a game and its genre
4. Recommend some random features based on the text similarity and closest tags

['jump', 'on', 'powerlines']
['VERB', 'ADP', 'NOUN']
['VB', 'IN', 'NNS']

['picks', 'up', 'in', 'the', 'titular', 'metropolis']
['VERB', 'ADP', 'ADP', 'DET', 'ADJ', 'NOUN']
['VBZ', 'RP', 'IN', 'DT', 'JJ', 'NN']

['bending', 'trials']
['VERB', 'NOUN']
['VBG', 'NNS']

